<a href="https://colab.research.google.com/github/MinjuKim0217/Python-Machine-Learning-Book/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EA%B5%90%EA%B3%BC%EC%84%9C_6%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 평가와 하이퍼파라미터 튜닝의 모범 사례

- 머신러닝 모델 성능 평가하기
- 머신러닝 알고리즘에서 일반적으로 발생하는 문제 분석하기
- 머신러닝 모델 세부 튜닝하기
- 여러가지 성능 지표를 사용하여 모델의 예측 성능 평가하기


# 파이프 라인을 사용한 효율적인 워크플로

데이터 압축을 위해 주성분 분석을 해봤음-> 이때 테스트 데이터셋에 있는 별도의 샘플처럼 새로운 데이터 스케일을 조정하고 압축하기 위해 훈련 데이터셋에서 학습한 파라미터를 재사용해야 한다.

**이때 유용하게 쓰이는것이 사이킷런의 Pipeline 클래스**


## 위스콘신 유방암 데이터셋

악성과 양성인 종양 세포 샘플 569개 들어가 있음

첫번째/두번째 열: 샘플의 고유 ID 번호, 진단 결과 (M=악성, B=양성)
3-32 번째 열: 세포 핵의 디지털 이미지에서 계산된 30개의 실수 값 특성



In [ ]:
# 코랩에서 실행할 경우 최신 버전의 사이킷런을 설치합니다.
!pip install --upgrade scikit-learn

In [ ]:
from IPython.display import Image

In [ ]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                 'machine-learning-databases'
                 '/breast-cancer-wisconsin/wdbc.data', header=None)

# UCI 머신 러닝 저장소에서 유방암 데이터셋을 다운로드할 수 없을 때
# 다음 주석을 해제하고 로컬 경로에서 데이터셋을 적재하세요:

# df = pd.read_csv('wdbc.data', header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
from sklearn.preprocessing import LabelEncoder

X=df.loc[:,2:].values #30개의 특성 X에 넣기
y=df.loc[:,1].values
le=LabelEncoder()
y=le.fit_transform(y) # 특성을 정수로 변환

In [ ]:
le.transform(['M','B'])

array([1, 0])

악성 종양은 클래스 1로, 양성 종양은 클래스 0으로

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= \
  train_test_split(X, y,
                   test_size=0.20, #테스트 데이터셋 20% 할당
                   stratify=y,
                   random_state=1)

## 파이프라인으로 변환기와 추정기 연결

위스콘신 데이터는 각각 다른 스케일로 이루어져 있기 때문에 스케일을 맞추어주어야 한다. -> 특성 표준화

여기서는 주성분 분석을 통해 초기 30차원에서 좀 더 낮은 2차원 부분 공간으로 데이터를 압축한다고 가정.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr=make_pipeline(StandardScaler(),
                       PCA(n_components=2),
                       LogisticRegression(random_state=1))

pipe_lr.fit(X_train,y_train)
y_pred=pipe_lr.predict(X_test)
print('테스트 정확도: %.3f' % pipe_lr.score(X_test, y_test))


테스트 정확도: 0.956


사이킷런의 Pipeline 클래스를 메타 추정기 (meta-estimator)나 개별 변환기와 추정기를 감싼 wrapper로 생각할 수 있다.

Pipeline  객체의 fit 메서드를 호출하면 데이터가 중간 단계에 있는 모든 변환기의 fit 메서트와 transform 메서드를 차례로 거쳐 추정기 객체레 도달한다. 

추정기는 변환된 훈련 데이터셋을 사용하여 학습한다 .

In [ ]:
import matplotlib.pyplot as plt
from sklearn import set_config
set_config(display='diagram')
pipe_lr

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])

In [ ]:
Image(url='https://git.io/JtsTr', width=500) 

# K-fold 교차 검증을 사용한 모델 성능 평가

- **홀드아웃 교차 검증**
- **K-Fold 교차 검증**

## 홀드아웃 방법

데이터셋을 모델 훈련에 사용할 **훈련 세트**와 일반화 성능을 추정하는데 사용할 **테스트 세트**로 나눈다. 

예측 성능을 높이기 위해 하이퍼파라미터를 튜닝하고 비교해야 하는데 이때 모델 선택에 같은 테스트 세트를 반복해서 재사용하면 이는 훈련 세트의 일부가 되고 모델이 과적합되는 원이 되낟. 

그로므로 데이터셋을 **훈련 세트**, **검증 세트**, **테스트 세트**로 나누자. 

검증세트를 이용하여 다른 하이퍼파라미터 값에서 모델을 훈련하는 것을 계속 반복하고 성능을 평가한 뒤, 만족할만한 성능이 나온 하이퍼파라미터를 이용하여 테스트 세트에서 모델의 일반화 성능을 추정한다 .


In [ ]:
Image(url='https://git.io/JtsTo', width=500) 

단, 홀드아웃 방법은 훈련 데이터르르 훈련 데이터셋과 검증 데이터셋으로 나누는 방법에 따라 성능 추정이 민감할 수 있다.

## K-Fold 교차검증

K-Fold 교차검증은 홀드아우셍 비해 훈련 세트의 분할에 덜 민감한 성능 추정을 얻을 수 있다. 

- 중복을 허락하지 않고 훈련 데이터셋을 K개의 폴드로 램덤하게 나눈 뒤, K-1개의 폴드로 모델을 훈련하고 나머지 하나의 폴드로 성능을 평가한다. 

- 이 과정을 K번 반복하여 K개의 모델과 성능 추정을 얻는다. 

- 만족할만한 성능이 나온 하이퍼파라미터를 찾은 후에는 전체 훈련 세트를 사용하여 모델을 다시 훈련하고 독립적인 테스트세트를 이용하여 최종 성능 추정을 한다. 


**훈련세트가 작다면 폴드 갯수를 늘리는 것이 좋다.**

**
K값이 증가하면 훈련 데이터가 더 여러번 반복해서 사용되고, 모델 성능을 평균하여 일반화 성능을 추정할 때 더 낮은 편향을 만든다.**

가장 최적의 K는 보통 10이지만 거대한 데이터셋을 쓸 땐 K가 5만 되어도 잘 실행된다. 

In [ ]:
Image(url='https://git.io/JtsT6', width=500) 

## 계층적 K-Fold 교차검증

데이터가 한쪽으로 편향되어 있을 경우 K-Fold 교차검증을 사용했을 때 성능 평가가 잘 되지 않을 수 있다. 

그럴때, 계층적 K-Fold 교차검증을 사용한다. 

계층적 K-Fold 교차검증은 각 폴드에서 클래스 비융이 전체 훈련 세트에 있는 클래스 비율을 대표하도록 유지.

**일반적으로 회귀에서는 K-Fold 교차검증을 사용하고 분류에는 StratifiedKFol**를 쓴다.

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print('폴드: %2d, 클래스 분포: %s, 정확도: %.3f' % (k+1,
          np.bincount(y_train[train]), score))
    
print('\nCV 정확도: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

폴드:  1, 클래스 분포: [256 153], 정확도: 0.935
폴드:  2, 클래스 분포: [256 153], 정확도: 0.935
폴드:  3, 클래스 분포: [256 153], 정확도: 0.957
폴드:  4, 클래스 분포: [256 153], 정확도: 0.957
폴드:  5, 클래스 분포: [256 153], 정확도: 0.935
폴드:  6, 클래스 분포: [257 153], 정확도: 0.956
폴드:  7, 클래스 분포: [257 153], 정확도: 0.978
폴드:  8, 클래스 분포: [257 153], 정확도: 0.933
폴드:  9, 클래스 분포: [257 153], 정확도: 0.956
폴드: 10, 클래스 분포: [257 153], 정확도: 0.956

CV 정확도: 0.950 +/- 0.014


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV 정확도 점수: %s' % scores)
print('CV 정확도: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV 정확도 점수: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV 정확도: 0.950 +/- 0.014


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=pipe_lr,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV 정확도 점수: %s' % scores)
print('CV 정확도: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV 정확도 점수: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV 정확도: 0.950 +/- 0.014


`cross_val_predict` 함수는 `cross_val_score`와 비슷한 인터페이스를 제공하지만 훈련 데이터셋의 각 샘플이 테스트 폴드가 되었을 때 만들어진 예측을 반환합니다. 따라서 `cross_val_predict` 함수의 결과를 사용해 모델의 성능(예를 들어, 정확도)을 계산하면 `cross_val_score` 함수의 결과와 다르며 바람직한 일반화 성능 추정이 아닙니다. `cross_val_predict` 함수의 사용 용도는 훈련 세트에 대한 예측 결과를 시각화하거나 7장에서 소개하는 스태킹(Stacking) 앙상블(Ensemble) 방법처럼 다른 모델에 주입할 훈련 데이터를 만들기 위해 사용합니다.

In [ ]:
from sklearn.model_selection import cross_val_predict

preds = cross_val_predict(estimator=pipe_lr,
                          X=X_train, 
                          y=y_train,
                          cv=10, 
                          n_jobs=-1)
preds[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

`method` 매개변수에 반환될 값을 계산하기 위한 모델의 메서드를 지정할 수 있습니다. 예를 들어 `method='predict_proba'`로 지정하면 예측 확률을 반환합니다. `‘predict’`, `‘predict_proba’`, `‘predict_log_proba’`, `‘decision_function’` 등이 가능하며 기본값은 `'predict'`입니다.

In [ ]:
from sklearn.model_selection import cross_val_predict

preds = cross_val_predict(estimator=pipe_lr,
                          X=X_train, 
                          y=y_train,
                          cv=10, 
                          method='predict_proba', 
                          n_jobs=-1)
preds[:10]

array([[9.93982352e-01, 6.01764759e-03],
       [7.64328337e-01, 2.35671663e-01],
       [9.72683946e-01, 2.73160539e-02],
       [8.41658121e-01, 1.58341879e-01],
       [9.97144940e-01, 2.85506043e-03],
       [9.99803660e-01, 1.96339882e-04],
       [9.99324159e-01, 6.75840609e-04],
       [2.12145074e-06, 9.99997879e-01],
       [1.28668437e-01, 8.71331563e-01],
       [7.76260670e-04, 9.99223739e-01]])